## Usage
Used to get intermediate from SBBGCA(including score, member, apis...)

## preRequirement
Have to run SBBGCAExample - startClustering() function and get two pickles first

In [1]:
tag = "allaple_woj_g_98"  # used for pickle name
outputPath = "output/SBBGCA_Clustering/allaple_woj_g_98/"

In [2]:
import pickle

# read the results from pickle files
with open(outputPath + tag + '_intermediate.pickle', 'rb') as handle:
    intermediate = pickle.load(handle)
with open(outputPath + tag + '_nameDict.pickle', 'rb') as handle:
    nameDict = pickle.load(handle)
    

In [3]:
import csv

descendant_dict = dict()
groupInfo_list = list()
groupMotif_dict = dict()
for key, value in intermediate.items():
    score = value[0]
    clusterName = value[1][0]
    memberSet = value[2]
    motifs = value[1][1]
    motifsSize = len(motifs)
    
    # merge descendants
    descendants = set() # a new set to save descendants
    for member in memberSet:
        if member[0] == "G": # if a cluster member is already a grouped cluster
            for descendant in descendant_dict[member] :
                descendants.add(descendant)
        else:
            descendants.add(member)

    descendant_dict[clusterName] = descendants
    descendantSize = len(descendant_dict)
    
    groupMotif_dict[clusterName] = motifs
    groupInfo_list.append((clusterName, score, memberSet, motifsSize))
    
# TODO: write file:  clusterName, score, members, motifCount
with open(outputPath + tag + "_GroupInfo.csv", 'w', newline='') as infoFile:
    spamwriter = csv.writer(infoFile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    header = ["ClusterName", "SimilarityScore", "Members", "MotifCount"]
    spamwriter.writerow(header)
    for group in groupInfo_list:
        spamwriter.writerow(group)
    
    
# TODO: write file:  clusterName, descendants list
with open(outputPath + tag + "_Descendants.csv", 'w', newline='') as desFile:
    spamwriter = csv.writer(desFile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    header = ["ClusterName", "DescendantSize", "Descendants"]
    spamwriter.writerow(header)
    for key in sorted(descendant_dict.keys(), key = lambda x : int(x[1::])):
        row = (key, len(descendant_dict[key]), descendant_dict[key])
        spamwriter.writerow(row)
        
# TODO: write file:  clusterName, MotifNumber, apis
with open(outputPath + tag + "_Motifs.csv", 'w', newline='') as motifFile:
    spamwriter = csv.writer(motifFile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    header = ["ClusterName", "MotifNumber", "APIs"]
    spamwriter.writerow(header)

    for key in sorted(groupMotif_dict.keys(), key = lambda x : int(x[1::])):
        group_motifs = groupMotif_dict[key]
        motifNumber = 0
        for motif in group_motifs:
            firstMotifAPI = True
            for api in motif[0]:
                if(firstMotifAPI):
                    row = (key, motifNumber, api)
                    firstMotifAPI = False
                else:
                    row = ("", "", api)
                spamwriter.writerow(row)
            motifNumber += 1